# TRT-35: Harmony STAC for Analysis Ready Data

### Notebook goal

Test a basic STAC catalog workflow from:
  *  Harmony 
  * CMR-STAC 
  * VEDA endpoint
  * External (e.g. Planetary Computer) STAC catalog?

to determine:
1. Interoperability with common stac libraries
2. Feature parity vs gaps in functionality for Harmony
3. Utility as input into VEDA workflow


## 1. Determine dataset/use case 
Resources: 
* https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html for CMR-STAC example
* https://nasa-openscapes.github.io/earthdata-cloud-cookbook/examples/LPDAAC/Find_and_Access_HLS_PointBuffer.html for another CMR-STAC example
* https://github.com/nasa/harmony-py/blob/main/examples/intro_tutorial.ipynb for Harmony STAC example
* https://nasa-impact.github.io/veda-docs/ for VEDA examples
* https://staging-stac.delta-backend.com/ : VEDA STAC API

We can use HLS example for CMR-STAC and a COG output example from Harmony to control for file format.

## 2. CMR-STAC Example
Copying from LP DAAC, VEDA, and Harmony tutorials above

What are the common STAC library imports across the three?

`from pystac_client import Client` is common across CMR-STAC and VEDA

**Questions for stakholders:** 
1. What is best practice when it comes to using `pystac_client`? 
2. Is Harmony's use of `pystac` incorrect? From 2i2c Hub I get an error importing STAC_IO so it is commented out below
3. Should we support both `pystac_client` and `intake`?

In [ ]:
# Imports needed from LP DAAC tutorial:

from shapely.geometry import Point
import json
import geopandas
import rasterio as rio
import rioxarray
import os
from pystac_client import Client 
import hvplot.xarray

# Imports needed from VEDA tutorial (https://nasa-impact.github.io/veda-docs/example-notebooks/open-and-plot.html):
from pystac_client import Client
import xarray as xr

import hvplot.xarray  # noqa

#Imports needed from Harmony
# Using PySTAC
from urllib.parse import urlparse
import requests
#from pystac import STAC_IO
from pystac import Catalog
# Using intake
import intake


### Set geometry for spatial search:

In [ ]:
field = geopandas.read_file('./data/ne_w_agfields.geojson')
roi = json.loads(field.to_json())['features'][0]['geometry']
roi

In [ ]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')

#New version of CMR (CMR-1.217.0-r23.1.1) deployed 2/22 now requires concept-id, not shortname. I still can't get this to work using collection-id however:
# HLSL30.v2.0 --> C2021957657-LPCLOUD
# HLSS30.v2.0 --> C2021957295-LPCLOUD
search = catalog.search(
    collections = "C2021957657-LPCLOUD",
#    collections = ["HLSL30.v2.0"],
#    intersects = roi,
    datetime = '2021-05/2021-08'
)               
search.matched()

In [ ]:
items = search.get_all_items()
print(items[0])
items[0].assets
# data_url = items[0].assets['B04'].href
# data_url

In [ ]:
# #We can do this with the latest version of earthaccess so that we don't have to use the GDAL driver code below. But, this needs to be tested first:  

# import earthaccess

# da = xr.open_mfdataset(earthaccess.open([data_url]), engine="rasterio")
# da = da.squeeze("band", drop=True)
# da

In [ ]:
rio_env = rio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

In [ ]:
da = rioxarray.open_rasterio(data_url)
da = da.squeeze('band', drop=True)
da

In [ ]:
da.hvplot.image(x='x', y='y', cmap='fire', rasterize=True, width=800, height=600, colorbar=True)    # colormaps -> https://holoviews.org/user_guide/Colormaps.html

## 3. VEDA Example


In [ ]:
STAC_API_URL = "https://staging-stac.delta-backend.com/"
collection = "social-vulnerability-index-overall-nopop"

In [ ]:
search = Client.open(STAC_API_URL).search(collections=[collection])
items = list(search.items())
items[0]

In [ ]:
s3_link = items[0].assets['cog_default'].href
print(s3_link)

# #I dont have access yet to the data:
# da = xr.open_dataset(s3_link, engine="rasterio")
# da = da.squeeze("band", drop=True)
# da

## 4. Harmony Example

In [ ]:
import sys
!{sys.executable} -m pip install -U harmony-py

Now need to restart kernel

In [ ]:
from harmony import BBox, Client, Collection, Request
from harmony.config import Environment
import datetime as dt

import intake

In [ ]:
harmony_client = Client() # assumes .netrc usage, option 3 above

In [ ]:
shapefile_path = './data/Big_Island_0005.zip' 

request = Request(
    collection=Collection(id='SENTINEL-1_INTERFEROGRAMS'),
    #spatial=BBox(-155.75, 19.26, -155.3, 19.94), # bounding box example that can be used as an alternative to shapefile input
    shape=shapefile_path,
    temporal={
        'start': dt.datetime(2020, 2, 24),
        'stop': dt.datetime(2020, 2, 25),
    },
    variables=['science/grids/data/unwrappedPhase'],
    format='image/tiff',
    max_results=2,
)

In [ ]:
job_id = harmony_client.submit(request)
job_id

In [ ]:
print(harmony_client.status(job_id))
harmony_client.wait_for_processing(job_id, show_progress=True)

In [ ]:
# urls = harmony_client.result_urls(job_id)
# list(urls)

In [ ]:
stac_catalog_url = harmony_client.stac_catalog_url(job_id)
stac_catalog_url

Before trying to use the same CMR-STAC or VEDA workflow above, we need to understand how this STAC URL compares to CMR-STAC as far as the STAC spec: According to https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html:

There are Four STAC Specifications:
*STAC Catalog (aka DAAC Archive)
*STAC Collection (aka Data Product)
*STAC Item (aka Granule)
*STAC API

**Given that the Harmony STAC URL contains items, does this mean that this is technically a STAC Collection URL?**

And according to:
https://pystac-client.readthedocs.io/en/stable/tutorials/pystac-client-introduction.html

"If the Catalog is an API, we have the ability to search for items based on spatio-temporal properties."

**So, we also need to confirm whether Harmony STAC endpoint is an API or not.** Based on the VEDA and CMR-STAC tutorials, those catalogs appear to be APIs as well. 



### Now try the above CMR-STAC workflow:

The commented line above gives me this error:

```ClientTypeError: Could not open Client (href=https://harmony.earthdata.nasa.gov/stac/111892c1-1f95-4be4-b278-031c41564a5c/?linktype=https), expected type=Catalog, found type=None```

In [ ]:
from pystac_client import Client 
#catalog = Client.open(stac_catalog_url)


I had to modify the existing Harmony-py tutorial to update based on the current `pystac` documentation (commented line is the old code:

In [ ]:
from urllib.parse import urlparse
import requests
from pystac.stac_io import StacIO
#from pystac import STAC_IO

def requests_read_method(uri):
    parsed = urlparse(uri)
    if parsed.hostname.startswith('harmony.'):
        return harmony_client.read_text(uri)
    else:
        return StacIO.default_read_text_method(uri)

StacIO.read_text_method = requests_read_method

The next code block from the Harmony-py tutorial throws this error:

```Exception: Could not read uri https://harmony.earthdata.nasa.gov/stac/111892c1-1f95-4be4-b278-031c41564a5c/?linktype=https

In [ ]:
# from pystac import Catalog

# cat = Catalog.from_file(stac_catalog_url)

# print(cat.title)
# s3_links = []
# for item in cat.get_all_items():
#     print(item.datetime, [asset.href for asset in item.assets.values()])
#     s3_links.append([asset.href for asset in item.assets.values()])

This recent Pangeo discourse post may be a hint, as Tom A. mentions that this could be a difference between the API and a static catalog: https://discourse.pangeo.io/t/pystac-client-cannot-load-stac-catalog/3071

```import pystac
pystac.read_file("https://meeo-s5p.s3.amazonaws.com/catalog.json")```

I get an error when I try with the Harmony URL:

In [ ]:
import pystac
#pystac.read_file(stac_catalog_url)